In [1]:
!pip install deltalake
!pip install duckdb

  Using cached deltalake-0.20.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.9 MB)
  Using cached pyarrow-18.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.0 MB)
  Using cached duckdb-1.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.1 MB)


In [10]:
!pip install pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 5.3 MB/s eta 0:00:0000:0100:01
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)


In [2]:
import duckdb
from deltalake.writer import write_deltalake

<jemalloc>: Out-of-range conf value: narenas:0


In [4]:
storage_options = {
    "AWS_REGION":'',
    'AWS_ACCESS_KEY_ID': '',
    'AWS_SECRET_ACCESS_KEY': '',
    'AWS_SESSION_TOKEN': '',
    'AWS_S3_ALLOW_UNSAFE_RENAME': 'true'
}

In [3]:
duckdb.sql(" \
CREATE SECRET my_ephem_sts_token ( \
    TYPE S3,\
    KEY_ID '',\
    SECRET '',\
    SESSION_TOKEN '',\
    REGION ''\
);").show()

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘



<jemalloc>: Out-of-range conf value: narenas:0


In [3]:
duckdb.sql("\
  SELECT * \
    FROM \
      parquet_schema( \
        's3://upgraded-pancake/soumik/superset-deltaduck/promo-1/source-data/green_tripdata_2024-01.parquet' \
        );\
").show()

┌────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────┬────────────┬─────────────┬─────────────────┬──────────────┬──────────────────┬───────┬───────────┬──────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                             file_name                                              │         name          │    type    │ type_length │ repetition_type │ num_children │  converted_type  │ scale │ precision │ field_id │                                            logical_type                                             │
│                                              varchar                                               │        varchar        │  varchar   │   varchar   │     varchar     │    int64     │     varchar      │ int64 │   int64   │  int64   │                                               varchar                                

In [19]:
jan_green_cab_trips = duckdb.sql("SELECT  \
      'green_cab' as cab_type, \
      2024 as year, \
      'January' as month, \
      ndf.* \
    FROM \
      's3://upgraded-pancake/soumik/superset-deltaduck/promo-1/source-data/green_tripdata_2024-01.parquet' as ndf;")

In [20]:
jan_green_cab_trips.limit(1).show();

┌───────────┬───────┬─────────┬──────────┬──────────────────────┬───────────────────────┬────────────────────┬────────────┬──────────────┬──────────────┬─────────────────┬───────────────┬─────────────┬────────┬─────────┬────────────┬──────────────┬───────────┬───────────────────────┬──────────────┬──────────────┬───────────┬──────────────────────┐
│ cab_type  │ year  │  month  │ VendorID │ lpep_pickup_datetime │ lpep_dropoff_datetime │ store_and_fwd_flag │ RatecodeID │ PULocationID │ DOLocationID │ passenger_count │ trip_distance │ fare_amount │ extra  │ mta_tax │ tip_amount │ tolls_amount │ ehail_fee │ improvement_surcharge │ total_amount │ payment_type │ trip_type │ congestion_surcharge │
│  varchar  │ int32 │ varchar │  int32   │      timestamp       │       timestamp       │      varchar       │   int64    │    int32     │    int32     │      int64      │    double     │   double    │ double │ double  │   double   │    double    │  double   │        double         │    double    │ 

write_deltalake(
    table_or_uri='s3://upgraded-pancake/soumik/superset-deltaduck/promo-1/deltalake/tbl_nyc_taxi_trips',
    storage_options=storage_options,
    data=jan_green_cab_trips.df(),
    mode='append',
    name='tbl_nyc_taxi_trips'
)

### Load 2nd file with Yellow Taxi Data for Jan, 2024

In [5]:
jan_yellow_cab_trips = duckdb.sql("SELECT  \
      'yellow_cab' as cab_type, \
      2024 as year, \
      'January' as month, \
      ndf.* \
    FROM \
      's3://upgraded-pancake/soumik/superset-deltaduck/promo-1/source-data/yellow_tripdata_2024-01.parquet' as ndf;")

In [6]:
jan_yellow_cab_trips.limit(1).show();

┌────────────┬───────┬─────────┬──────────┬──────────────────────┬───────────────────────┬─────────────────┬───────────────┬────────────┬────────────────────┬──────────────┬──────────────┬──────────────┬─────────────┬────────┬─────────┬────────────┬──────────────┬───────────────────────┬──────────────┬──────────────────────┬─────────────┐
│  cab_type  │ year  │  month  │ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datetime │ passenger_count │ trip_distance │ RatecodeID │ store_and_fwd_flag │ PULocationID │ DOLocationID │ payment_type │ fare_amount │ extra  │ mta_tax │ tip_amount │ tolls_amount │ improvement_surcharge │ total_amount │ congestion_surcharge │ Airport_fee │
│  varchar   │ int32 │ varchar │  int32   │      timestamp       │       timestamp       │      int64      │    double     │   int64    │      varchar       │    int32     │    int32     │    int64     │   double    │ double │ double  │   double   │    double    │        double         │    double    │        double 

In [ ]:
write_deltalake(
    table_or_uri='s3://upgraded-pancake/soumik/superset-deltaduck/promo-1/deltalake/tbl_nyc_taxi_trips',
    storage_options=storage_options,
    data=jan_yellow_cab_trips.df(),
    mode='append',
    name='tbl_nyc_taxi_trips'
)